### Method 1

In [ ]:
for _ in range(10):
    for d_name in ["rep-1", "rep-2"]:
        # Get handle to the each deployment and invoke its method.
        # Which replica the request is dispatched to is determined
        # by the Router actor.
        handle = serve.get_deployment(d_name).get_handle()
        print(f"handle name : {d_name}")
        print(f"prediction  : {ray.get(handle.remote(random()))}")
        print("-" * 2)

### Method 2

In [ ]:
for _ in range(10):
    for d_name in ["rep-1", "rep-2"]:
        # Send HTTP request along with data payload
        url = f"http://127.0.0.1:8000/{d_name}"
        print(f"handle name : {d_name}")
        print(f"prediction  : {requests.get(url, params={'data': random()}).text}")